# ELE6310 - Assignment 2 - Data-Flow and Design Space

#### Name: 
#### Student ID: 

In [ ]:
#@title Mount your Google Drive
%matplotlib inline
%load_ext autoreload
%autoreload 2

from google.colab import drive
drive.mount('/content/gdrive')


In [ ]:
#@title Link your assignment folder & install requirements
#@markdown Enter the path to the assignment folder in your Google Drive
import sys
import os
import shutil
import warnings

folder = "/content/gdrive/MyDrive/ELE6310/A2" #@param {type:"string"}
!ln -Ts $folder /content/A2 2> /dev/null

# Add the assignment folder to Python path
if '/content/A2' not in sys.path:
    sys.path.insert(0, '/content/A2')

# Install requirements
!pip install -qr /content/A2/requirements.txt

# Check if CUDA is available
import torch
if not torch.cuda.is_available():
    warnings.warn('CUDA is not available.')

## 1- Structured Pruning

* First, complete `model_to_spars` and `generate_resnet_layers` in `solution. py`.

In [ ]:
import solution
import torch
import numpy as np
import random
from matplotlib import pyplot as plt
import os

In [ ]:
from common.utils import load_CIFAR10_dataset, evaluate, fit, model_size
from common.resnet import resnet20

In [ ]:
Seed = 6310
torch.manual_seed(Seed)
np.random.seed(Seed)
random.seed(Seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(Seed)
    torch.cuda.manual_seed_all(Seed)

In [ ]:
train_loader, test_loader, calibration_loader = load_CIFAR10_dataset(batch_size=256, calibration_batch_size=1024)
model = resnet20(pretrained=True, save_path='./save/')
device = torch.device('cuda:0')
model.to(device)

accuracy = evaluate(model, test_loader, device)
print("test accuracy of fp model:", accuracy, "model size:", model_size(model))


* Use a method of your choice to find the optimal energy consumption for ResNet-20 with a constraint on test accuracy above 85%.

Any reasonable attempt at exploring the design space will give you full marks. Better approaches/results will be considered for bonus points.

In [ ]:
prune_ratio_dict = {
    'conv1': 0.3,
    'layer1.0.conv1': 0.3,
    'layer1.0.conv2': 0.3,
    'layer1.1.conv1': 0.3,
    'layer1.1.conv2': 0.3,
    'layer1.2.conv1': 0.3,
    'layer1.2.conv2': 0.3,
    
    'layer2.0.conv1': 0.3,
    'layer2.0.conv2': 0.3,
    'layer2.0.downsample.0': 0.3,
    'layer2.1.conv1': 0.3,
    'layer2.1.conv2': 0.3,
    'layer2.2.conv1': 0.3,
    'layer2.2.conv2': 0.3,
    
    'layer3.0.conv1': 0.3,
    'layer3.0.conv2': 0.3,
    'layer3.0.downsample.0': 0.3,
    'layer3.1.conv1': 0.3,
    'layer3.1.conv2': 0.3,
    'layer3.2.conv1': 0.3,
    'layer3.2.conv2': 0.3
}


In [ ]:

sparsed_model = solution.model_to_spars(model, prune_ratio_dict)
accuracy = evaluate(sparsed_model, test_loader, device)
print("test accuracy after pruning:", accuracy, "model size:", model_size(sparsed_model))


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(sparsed_model.parameters(), 1e-3, momentum=0.9, weight_decay=0.0005, nesterov=True)
scheduler = None
train_accuracy, test_accuracy = fit(sparsed_model, 5, train_loader, test_loader, criterion, optimizer, scheduler, device)

After fine-tuning, save the model and generate the YAML files for each layers of the pruned network. Then you can use `run_Accelergy` to estimate the energy consumption of pruned network.

In [ ]:
solution.generate_resnet_layers(sparsed_model, base_path='common/layer_prob_base.yaml',  path='Q3/prob')